# Brute-Force SPCA
This notebook implements the brute-force Sparse PCA algorithm. We compare performance across different values of sparsity `k` and matrix dimensions `p`.


In [ ]:
pip install joblib

In [ ]:
import numpy as np
import math
from itertools import combinations
from tqdm import tqdm
import matplotlib.pyplot as plt
from joblib import Parallel, delayed



In [ ]:
# Brute-force Sparse PCA with parallelization
def brute_force_spca(orig_matrix, k):
    p = orig_matrix.shape[0]
    max_val = 0
    best_subset = None

    def process_subset(subset):
        submatrix = orig_matrix[np.ix_(subset, subset)]
        eigval = np.linalg.norm(submatrix, ord=2)
        return eigval, subset

    subsets = combinations(range(p), k)

    # Parallelize the subset processing
    results = Parallel(n_jobs=-1)(delayed(process_subset)(subset) for subset in tqdm(subsets, total=math.comb(p, k)))

    # Find the max eigenvalue and best subset
    for eigval, subset in results:
        if eigval > max_val:
            max_val = eigval
            best_subset = subset

    return max_val, best_subset

In [ ]:
# Set up synthetic matrix
np.random.seed(42)
K = 30
p = 10
D = np.random.randn(K, p)
D /= np.linalg.norm(D, axis=1, ord=2, keepdims=True)
B = D @ D.T
B /= np.linalg.norm(B, ord=2)

# Run brute-force SPCA for different k
results = []

MAX_FEASIBLE_K = 30

for k in range(1, K + 1):
    if k <= MAX_FEASIBLE_K:
        val, _ = brute_force_spca(B, k)
    else:
        val = np.nan
    results.append(val)


 45%|████▍     | 13469692/30045015 [16:30<19:25, 14223.64it/s]

In [ ]:
# Plotting the results
plt.plot(range(1, K + 1), results, label="Brute Force SPCA")
plt.xlabel('Sparsity (k)')
plt.ylabel('Max Eigenvalue')
plt.title('Sparse PCA Performance')
plt.grid(True)
plt.legend()
plt.show()